# TL;DR:
This notebook compares the previously used gaussian distribution of P_hum on latitudes with the AHF data used in 

1. Flanner2009 via Energy_Consumption * population
2. Chen2016 Nighttime_light 

Then the gaussian distribution is adjusted to match better with Flanner's Data (Disclaimer: No statistical method/analysis, just visually)

Result: Both data sets inhibit more AHF in equator near latitudes (especially in the nighttime_lights data). 

### EDIT 25.1.
FITTED GAUSS IN CHAPTER IMPROVE GAUSS

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
fs=20 #fontsize for labels, legend, ...\n",
font={'family' : 'serif', 'size' : fs} #define font for legend\n",
plt.rc('font', **font)

In [ ]:
def area_mean(_lats, x):
    weights=np.cos(_lats*np.pi/180)
    return sum(weights*x)/sum(weights)

# Flanner 2009

### Load P-hum Data from Flanner 2009 
http://www.cgd.ucar.edu/tss/ahf/data/

- grid: 0.5x0.5 degrees for lat=[-90..90], lon=[-180, 180]
- AHF (lat, lon)= Energy_consumption_in_country * population(lat,lon)
- Time Evolution: 2005 -> 2040: EIA estimates for Energy consumption (and population? TODO); afterwards 2% growth

In [ ]:
#data=xr.open_dataset("AHF_2005-2040_0.5x0.5.nc")
data=xr.open_dataset("/path_to_file/AHF_2005-2040_0.5x0.5.nc")
##########################################################
####   THIS FILE IS FROM FLANNER2009   ###################
##########################################################

### Gauss Verteilung von P_hum used in previous plots

In [ ]:
gauss_breite=15
mlat=45
P_glob=0.034
a=P_glob*sum(np.cos(data.lat*np.pi/180)) / (sum(np.cos(data.lat*np.pi/180)* np.exp(-1./(2*gauss_breite**2) * (data.lat-mlat)**2)))
P_hum_dist=a*np.exp(-1./(2*gauss_breite**2) * (data.lat-mlat)**2)
test_P_glob=area_mean(data.lat, P_hum_dist)

### Average the longitude values 

In [ ]:
data_lat=data.variables['AHF'].mean(dim='lon')

### Then average over the latitudes (including weights)  to get glob averages $\overline{AHF}(t)$

In [ ]:
glob_mean_AHF=np.zeros([len(data.time)])
for i in range(0,len(data.time)):
    glob_mean_AHF[i]=area_mean(data.lat, data_lat[i,:])

In [ ]:
fig=plt.figure(figsize=(16,5))
ax=fig.add_subplot(121)
ax2=fig.add_subplot(122)
ax2.plot(data.lat, data_lat[10,:], label='Flanner: year 2015, P_glob_Flanner='+'%.3f' % glob_mean_AHF[10])
ax2.plot(data.lat, data_lat[-1,:], label='Flanner: year 2040, P_glob_Flanner='+'%.3f' % glob_mean_AHF[-1])
years=np.linspace(2005, 2040, num=36)
ax.plot(years, glob_mean_AHF, '-g', label='Flanner: EIA projection')
ax.plot(years[10:], [0.034*1.02**i for i in range(0,len(years[10:]))] , '--g', label='2% growth/a')
ax.legend()
ax.set_xlabel("years")
ax.set_ylabel("P_hum (glob mean) W/m^2")
ax2.set_xlabel("lats")
ax2.set_ylabel("P_hum (lon mean) W/m^2")
ax2.plot(data.lat, P_hum_dist, '--k', lw=2, label='Gauss with P_glob=0.034')
ax2.legend()

## Improve Gauss

### Brute Force

In [ ]:
gauss_breite=10
P_glob=0.034
mlat=38
a=P_glob*sum(np.cos(data.lat*np.pi/180)) / (sum(np.cos(data.lat*np.pi/180)* np.exp(-1./(2*gauss_breite**2) * (data.lat-mlat)**2)))
P_hum_dist=a*np.exp(-1./(2*gauss_breite**2) * (data.lat-mlat)**2)
test_P_glob=area_mean(data.lat, P_hum_dist)

In [ ]:
fig=plt.figure(figsize=(16,4))
ax=fig.add_subplot(111)
ax.plot(data.lat, data_lat[10,:], label='Flanner: year 2015, P_glob_Flanner='+'%.3f' % glob_mean_AHF[10])
ax.plot(data.lat, P_hum_dist, '--k', lw=2, label='Gauss with P_glob=0.034')


### Attempt to make the comparison easier: Cut-Off high frequencies --> smooth Flanner curve

In [ ]:
import scipy.fftpack

In [ ]:
w = scipy.fftpack.rfft(data_lat[10,:])
cutoff_idx = np.linspace(0,len(w), num=len(w)) >40 #40 is random
w2=w.copy()
w2[cutoff_idx]=0
plt.plot(w, '-b', label='more Frequencies used for detailed plots above')
plt.plot(w2, '-y', label='fewer Frequencies used for plot below')
plt.legend()

In [ ]:
data_new=scipy.fftpack.irfft(w2)
print(area_mean(data.lat, data_new))
plt.figure(figsize=(16,4))
plt.plot(data.lat, data_new, label='Flanner smoothed: P_glob='+'%.3f' % area_mean(data.lat, data_new))
plt.plot(data.lat, P_hum_dist, '--k', lw=2, label='Gauss with P_glob=0.034')
plt.legend()

## Fazit:

Nach Anpassung der Mittel-Latitude von 45 zu 40 Grad und der Gauss Breite von 15 zu 10 Grad, wird der Peak gut abgebildet.

Nicht berücksichtigt sind AHF-Beiträge von Längengräden um den Äquator. 



## FIT GAUSS

In [ ]:
import scipy.optimize as opt
def f(phi, a, b, c):
    return a*np.exp(-(phi-b)**2/c**2)
popt2015, pcov2015=opt.curve_fit(f,data.lat,data_lat[10,:], bounds=([0,25, 1], [2, 60, 100]))
print(popt2015)
plt.plot(data.lat, data_lat[10,:])
plt.plot(data.lat, f(data.lat, *popt2015), '--r')
### normalize to get P_glob:
fit=f(data.lat, *popt2015)
fit_glob=area_mean(data.lat, fit)
fit_normalized=P_glob/fit_glob*fit
plt.plot(data.lat, fit_normalized, '-g')

In [ ]:
popt2040, pcov2040=opt.curve_fit(f,data.lat,data_lat[-1,:], bounds=([0,25, 1], [2, 60, 100]))
print(popt2040)
ax=plt.figure().add_subplot(111)
ax2=ax.twinx()
ax.plot(data.lat, data_lat[-1,:])
fit2040=f(data.lat, *popt2040)
fit_glob40=area_mean(data.lat, fit2040)
fit_normalized2040=P_glob/fit_glob40*fit2040
ax.plot(data.lat, 1.02**25 * fit_normalized2040, '-m')
ax.plot(data.lat, fit_normalized, '--c')
ax2.plot(data.lat, fit_normalized2040-fit_normalized, '--k')
### normalize to get P_glob:
fit=f(data.lat, *popt2040)

In [ ]:
y=P_glob/fit_glob*f(data.lat, *popt2015)
p,c=opt.curve_fit(f, data.lat, y)
plt.plot(data.lat, y)
plt.plot(data.lat, f(data.lat, *p))
print(p)

## Small Problem:
expected growth (up to 2040) especially at 25N 

In [ ]:
w = scipy.fftpack.rfft(data_lat[-1,:])
cutoff_idx = np.linspace(0,len(w), num=len(w)) >40 #40 is random
w2=w.copy()
w2[cutoff_idx]=0
data_new_2040=scipy.fftpack.irfft(w2)

In [ ]:
print("P_hum_glob in 2040: ", area_mean(data.lat, data_new_2040))
plt.figure(figsize=(16,4))
plt.plot(data.lat, data_new, label='Flanner smoothed: P_glob='+'%.3f' % area_mean(data.lat, data_new))
plt.plot(data.lat, data_new_2040, label='Flanner 2040 smoothed: P_glob ='+'%.3f' % area_mean(data.lat, data_new_2040))
plt.plot(data.lat, P_hum_dist*1.02**25, '--k', lw=2, label='Gauss 2040 with P_glob='+'%.3f' % area_mean(data.lat, P_hum_dist*1.02**25))
plt.legend()

# Chen2016

In [ ]:
import tifffile as tif

Data for 2010 nighttime light of satelite Nr. 18:
- cloud free
- fire light removed
- background removed
- grid: 30 arc sec for lat=[-65..75], lon=[-180..180]

Note: Some locations have only few cloud-free nights and hence less available data.

In [ ]:
#image=tif.imread("F182010.v4d_web.stable_lights.avg_vis.tif")
image=tif.imread("/home/peter/PIK_LargeData/F182010.v4d_web.stable_lights.avg_vis.tif")

In [ ]:
print("latitude=(65+75)degrees=140*3600/30 pixel_for_30_arcsec=16800 pixel")
print("longitude=(180*2)degrees=360*3600/30 pixel_for_30_arcsec=43200 pixel")
print(np.shape(image))

### Take the average over all longitudes

In [ ]:
lats=np.linspace(-65, 75, num=len(image[:,0]))
image_lats=0*lats
for i in range(0,len(image_lats)):
    image_lats[-i]=np.mean(image[i,:]) ## Note: the light-data starts at North Pole (i guess), hence image_lats[-i]
plt.plot(lats, image_lats)

In [ ]:
#Plot the maximum values in each latitude band
#lats=np.linspace(-65, 75, num=16801)
#image_max=0*image[:,0]
#for i in range(0,len(image_lats)):
#    image_max[i]=max(image[:,i])
#plt.plot(lats, image_max)
#plt.title("Max value of image (=63) for each latitude")

### Add and insert light=0 for the rest of the lats.

In [ ]:
nighttime_lights_short=np.array(image_lats)
south_extension=np.array([0. for s in range(0,120) for deg in range(-90, -65)])
north_extension=np.array([0. for s in range(0,120) for deg in range(75, 90)])
print('Test of dimensions:', np.shape(south_extension), '+', np.shape(nighttime_lights_short),
      '+',np.shape(north_extension),  "=??= 180*120 = 21600")
nighttime_lights=np.concatenate((south_extension,nighttime_lights_short))
nighttime_lights=np.concatenate((nighttime_lights,north_extension))
lats_light=np.linspace(-90, 90, num=len(nighttime_lights))

### normalize to 0.034 W/m² 

In [ ]:
P_glob=0.034
tot_light=area_mean(lats_light, nighttime_lights)
rel_lights=nighttime_lights/tot_light
P_lights=P_glob*rel_lights

In [ ]:
plt.plot(lats_light, P_lights, '-y', label='P_hum_lights [W/m^2]')
plt.legend()

In [ ]:
fig=plt.figure(figsize=(16,9))
ax=fig.add_subplot(111)
ax.plot(data.lat, data_lat[10,:], '-r',  label='Flanner: year 2015, P_glob_Flanner='+'%.3f' % area_mean(data.lat, data_lat[10,:]))
ax.plot(lats_light, P_lights, '-y' , label="Chen(1 yr and 1 satelite only): nighttime lights normalized to P_glob="+'%.3f' % area_mean(lats_light, P_lights))
ax.set_xlabel("lats")
ax.set_ylabel("P_hum (lon mean) W/m^2")
ax.plot(data.lat, P_hum_dist, '--k', lw=2, label='Gauss with P_glob='+'%.3f' % area_mean(data.lat, P_hum_dist))
ax.legend()

# Fazit

### In the longitudinal mean, nighttime light data in Chen and Flanners Energy consumption (weighted by population denisty) inhibit the same features and yield the same distribution with only tiny deviations. The Gauss distribution (after adjusting the center and width) can roughly reproduce the distribution with certain flaws in (1) the time evolution, (2) underrepresentation of Equator regions and (3) missing minor peaks (e.g.55N).

### For high latitudes the light data might be exaggerated since people might install more light sources to compensate for the missing daylight in winter time.

### FIT GAUSS TO CHEN

In [ ]:
fs=19    #fontsize for labels, legend, ...\n",
font={'family' : 'serif', 'size' : fs}  #define font for legend\n",
plt.rc('font', **font)

In [ ]:
import scipy.optimize as opt
def f(phi, a, b, c):
    return a*np.exp(-(phi-b)**2/c**2)
poptchen, pcovchen=opt.curve_fit(f,lats_light,P_lights, bounds=([0,25, 1], [2, 60, 100]))
print(popt2015)
print(poptchen)
#plt.plot(lats_light, P_lights)
### normalize to get P_glob:
fit_chen=f(lats_light, *poptchen)
fit_glob_chen=area_mean(lats_light, fit_chen)
fit_n_chen=P_glob/fit_glob_chen*fit_chen
#plt.plot(data.lat, fit_normalized, '-g')
#plt.plot(lats_light, fit_n_chen, '--r')

fig=plt.figure(figsize=(16,8))
#plt.yticks([])
#plt.xticks([])
ax0=fig.add_subplot(111)
ax0.spines['top'].set_color('none')
ax0.spines['bottom'].set_color('none')
ax0.spines['left'].set_color('none')
ax0.spines['right'].set_color('none')
ax0.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')

ax=fig.add_subplot(211)
ax2=fig.add_subplot(212, sharex=ax, sharey=ax)
ax.set_xlim(-90, 90)
ax.plot(data.lat, data_lat[10,:], '-r',  
        label=r'Flanner (2009): $\overline{P_{\rm hum}}=$'+'%.3f' % area_mean(data.lat, data_lat[10,:])
        +r'$\, \frac{ \rm W}{\rm m^2}$')
ax2.plot(lats_light, P_lights, '-y' , 
         label=r"Chen et al. (2016): Nighttime lights normalized to $\overline{P_{\rm hum}}=$"+
         '%.3f' % area_mean(lats_light, P_lights)+r'$\, \frac{\rm W}{ \rm m^2}$')
plt.setp(ax.get_xticklabels(), visible=False)
ax2.annotate(r"(One year (2010) "+'\n'+"one satelite only)", xy=(-80,0), xytext=(-80,0.1),
             arrowprops={'facecolor':'yellow'}, fontsize=16)
ax.plot(data.lat, fit_normalized, '--r', lw=4, label=r'Gauss with $\overline{P_{\rm hum}}=$'+'%.3f' 
        % area_mean(data.lat, P_hum_dist)+r'$\, \frac{\rm W}{\rm m^2}$'+' fitted to Flanner (2009)')
ax2.plot(lats_light, fit_n_chen, '--y', lw=4, label=r'Gauss with $\overline{P_{\rm hum}}=$'+'%.3f' 
         % area_mean(data.lat, P_hum_dist)+r'$\, \frac{\rm W}{\rm m^2}$'+' fitted to Chen et al. (2016)')
ax.legend(fontsize=16, loc='upper left')
ax2.legend(fontsize=16, loc='upper left')
ax.grid()
ax2.grid()
ax0.set_xlabel(r"$\theta$ - latitudes")
ax0.set_ylabel(r"$P_{\rm hum}(\theta) \ [{\rm W/m^2}]$") #averaged over longitudes)")

gausschen=r"$0.14\, \rm{\frac{W}{m^2}} \cdot \exp{(-\left(\frac{\theta-42.17^\circ}{18.54^\circ}\right)^2)}$"
ax2.annotate(gausschen, xy=(-40,0), xytext=(-40,0.1), arrowprops={'facecolor':'yellow'}, fontsize=23)
gaussflanner=r"$0.16\, \rm{\frac{W}{m^2}} \cdot \exp{(-\left(\frac{\theta-39.68^\circ}{16.64^\circ}\right)^2)}$"
ax.annotate(gaussflanner, xy=(-40,0), xytext=(-40,0.1), arrowprops={'facecolor':'red'}, fontsize=23)

plt.tight_layout()
plt.savefig("P_hum_Flanner_Chen.png")


Result: Chen is a bit more northern and a bit broader...